<a href="https://colab.research.google.com/github/coryskeers/dl_phones/blob/master/g2p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b.symbols
!wget http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b.phones
!wget http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b

# Build our phoneme descriptions
with open('cmudict-0.7b.phones') as phonefile:
  phone_defs = {}
  for line in phonefile:
    split = line.split()
      phone_defs[split[0]] = split[1]

# Get all symbol combos and create an indexer
with open('cmudict-0.7b.symbols') as symfile:
  phone_index = {}
  index_phone = {}
  i = 0
  for line in symfile:
    phone_index[line.strip()] = i
    index_phone[i] = line.strip()
    i += 1



--2020-02-19 02:07:40--  http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b.symbols
Resolving svn.code.sf.net (svn.code.sf.net)... 216.105.38.17
Connecting to svn.code.sf.net (svn.code.sf.net)|216.105.38.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281 [text/plain]
Saving to: ‘cmudict-0.7b.symbols’

cmudict-0.7b.symbol 100%[===================>]     281  --.-KB/s    in 0s      

2020-02-19 02:07:40 (51.9 MB/s) - ‘cmudict-0.7b.symbols’ saved [281/281]

--2020-02-19 02:07:41--  http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict-0.7b.phones
Resolving svn.code.sf.net (svn.code.sf.net)... 216.105.38.17
Connecting to svn.code.sf.net (svn.code.sf.net)|216.105.38.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382 [text/plain]
Saving to: ‘cmudict-0.7b.phones’

cmudict-0.7b.phones 100%[===================>]     382  --.-KB/s    in 0s      

2020-02-19 02:07:41 (63.8 MB/s) - ‘cmudict-0.7b.phones’ saved [382/382